# Proyecto MusicStream – Año 2016

Extracción de datos desde Spotify y Last.fm usando funciones separadas.
Lógica clara, modular y fácil de explicar.

## 1. Importaciones y configuración

In [2]:
# Extracción de datos 
# --- 1. SISTEMA Y SEGURIDAD ---
# Gestiona variables de entorno y control de pausas
from dotenv import load_dotenv       # Carga el archivo .env con claves secretas
load_dotenv()                        # Inyecta las claves como variables de entorno
import os                              # Accede a las claves con os.environ.get
import time                            # Permite pausas controladas para no saturar APIs

# --- 2. EXTRACCIÓN Y MANEJO DE APIS ---
# Comunicación y autenticación con APIs de música
import requests                        # Peticiones HTTP genéricas (ej. Last.fm)
import spotipy                          # Wrapper para la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials  # OAuth seguro de Spotify
from spotipy.exceptions import SpotifyException      # Manejo de errores de Spotify (ej. 429)

# --- 3. ESTRUCTURA Y TRANSFORMACIÓN DE DATOS ---
import pandas as pd                     # DataFrames para limpiar, unir y manipular datos

# --- 4. NORMALIZACIÓN Y UTILIDADES ---
from urllib.parse import quote           # Codifica URLs para nombres con espacios/acentos


In [25]:
# Conexión a sql
import mysql.connector
from mysql.connector import Error # Manejo de errores específicos de MySQL

# --- Otras Utilidades ---
import numpy as np # Necesario para manejar np.nan (valores nulos) al insertar DataFrames


## 2. Credenciales: Spotify

In [4]:
# --- COMPROBACIÓN Y CONEXIÓN CON SPOTIFY ---
# Recupera las credenciales del archivo .env
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# Verifica que ambas credenciales estén disponibles
if not client_id or not client_secret:
    print("⚠️ ADVERTENCIA: Las credenciales de Spotify no están cargadas. Revisa tu archivo .env.")
else:
    # Configura la autenticación segura con Spotify usando OAuth de aplicación
    mis_credenciales = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    # Inicializa el cliente de Spotify con las credenciales
    spotify = spotipy.Spotify(auth_manager=mis_credenciales)
    print("✅ Conexión con Spotify inicializada.")


✅ Conexión con Spotify inicializada.


In [5]:
# --- PRUEBA DE CONEXIÓN CON SPOTIFY ---
# Objetivo: comprobar que el cliente de Spotify funciona y devuelve datos válidos.

# Realizamos una búsqueda de artistas por género (genre solo funciona con artistas)
prueba = spotify.search(
    q='genre:latin',   # Buscamos artistas del género "latin"
    type='artist',     # Tipo de búsqueda: artistas
    limit=1            # Solo 1 resultado, suficiente para probar
)

# Extraemos el primer artista de los resultados
artista = prueba['artists']['items'][0]

# Mostramos el nombre del artista para confirmar que la conexión y la búsqueda funcionan
print("✅ Conexión exitosa con Spotify")
print("🎵 Artista de prueba encontrado:", artista['name'])

✅ Conexión exitosa con Spotify
🎵 Artista de prueba encontrado: Bad Bunny


In [6]:
# --- COMPROBACIÓN DE LA CLAVE DE LAST.FM ---
# Recupera la API key desde el archivo .env
api_key_lastfm = os.getenv("API_KEY_LASTFM")

# Verifica que la clave exista antes de usarla
if not api_key_lastfm:
    print("⚠️ ADVERTENCIA: La clave de Last.fm no está cargada. Revisa tu archivo .env.")
else:
    # Confirmamos que la clave está disponible para futuras peticiones
    print("✅ Conexión con Last.fm inicializada.")

✅ Conexión con Last.fm inicializada.


In [7]:
# --- PRUEBA DE CONEXIÓN BÁSICA CON LAST.FM ---
if not api_key_lastfm:
    print("❌ Clave API de Last.fm no definida.")
else:
    try:
        # Hacemos una petición mínima para verificar la conexión usando el método 'chart.getTopArtists'
        respuesta = requests.get(
            "http://ws.audioscrobbler.com/2.0/",
            params={
                'method': 'chart.gettopartists',
                'api_key': api_key_lastfm,
                'format': 'json',
                'limit': 1  # Solo necesitamos 1 artista para probar
            }
        )
        respuesta.raise_for_status()
        print("✅ Conexión exitosa con Last.fm")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al conectar con Last.fm: {e}")

✅ Conexión exitosa con Last.fm


## 3. Obtener información Spotify

In [8]:
# ────────────────────────────────────────────────
# FUNCIÓN SEGURA PARA HACER SEARCH
# ────────────────────────────────────────────────
def spotify_search_seguro(spotify, query, tipo, limite=10, reintentos=3):
    """
    Función que busca en Spotify de forma segura.
    Detecta el error 429 (rate limit) y espera si es necesario.
    
    Parámetros:
    - spotify: objeto de la API
    - query: texto a buscar
    - tipo: "artist" o "album"
    - limite: máximo resultados
    - reintentos: cuántas veces intentar si hay error 429
    """
    intento = 0  # Contador de intentos

    while intento < reintentos:
        try:
            # Llamada a Spotify
            return spotify.search(q=query, type=tipo, limit=limite)

        except SpotifyException as e:
            if e.http_status == 429:
                # Si nos dicen "espera", tomamos el tiempo de Retry-After
                espera = int(e.headers.get("Retry-After", 1))
                print(f"⏳ Demasiadas peticiones. Esperando {espera} segundos...")
                time.sleep(espera)
                intento += 1  # Sumamos intento
            else:
                print("❌ Error de Spotify:", e)
                return None

        except Exception as e:
            # Capturamos cualquier error inesperado
            print("❌ Error inesperado:", e)
            return None

    # Si agotamos todos los intentos
    print("⚠️ Demasiados intentos. Se omite esta búsqueda.")
    return None


In [9]:
# ────────────────────────────────────────────────
# FUNCIÓN SEGURA PARA OBTENER CANCIONES DE UN ÁLBUM
# ────────────────────────────────────────────────
def spotify_album_tracks_seguro(spotify, id_album, reintentos=3):
    """
    Función que obtiene las canciones de un álbum de forma segura.
    Maneja errores 429 y cualquier otro error sin que el programa se caiga.
    """
    intento = 0

    while intento < reintentos:
        try:
            return spotify.album_tracks(id_album)

        except SpotifyException as e:
            if e.http_status == 429:
                espera = int(e.headers.get("Retry-After", 1))
                print(f"⏳ Rate limit en álbum. Esperando {espera} segundos...")
                time.sleep(espera)
                intento += 1
            else:
                print("❌ Error de Spotify:", e)
                return None

        except Exception as e:
            print("❌ Error inesperado:", e)
            return None

    print("⚠️ Demasiados intentos con este álbum.")
    return None


In [10]:
# ────────────────────────────────────────────────
# FUNCIONES DE NEGOCIO (LÓGICA DEL PROBLEMA)
# ────────────────────────────────────────────────

def buscar_artistas_por_genero(spotify, genero):
    """
    Busca artistas de un género específico.
    """
    resultado = spotify_search_seguro(spotify, "genre:" + genero, "artist")
    if resultado is None:
        return []
    return resultado["artists"]["items"]  # Devolvemos la lista de artistas

In [11]:
def buscar_albumes_por_artista_y_año(spotify, nombre_artista, año):
    """
    Busca álbumes de un artista en un año concreto.
    """
    query = "artist:" + nombre_artista + " year:" + str(año)
    resultado = spotify_search_seguro(spotify, query, "album")
    if resultado is None:
        return []
    return resultado["albums"]["items"]  # Devolvemos solo los álbumes

In [12]:
def obtener_canciones_de_album(spotify, id_album):
    """
    Devuelve las canciones de un álbum usando la función segura.
    """
    resultado = spotify_album_tracks_seguro(spotify, id_album)
    if resultado is None:
        return []
    return resultado["items"]

In [13]:
# ────────────────────────────────────────────────
# PROCESAR TODAS LAS CANCIONES DE UN GÉNERO
# ────────────────────────────────────────────────
def procesar_genero(spotify, genero, año, albumes_ya_vistos):
    """
    Devuelve una lista de canciones de un género, sin duplicados de álbumes.
    """
    canciones = []

    artistas = buscar_artistas_por_genero(spotify, genero)
    for artista in artistas:
        nombre_artista = artista["name"]

        albumes = buscar_albumes_por_artista_y_año(spotify, nombre_artista, año)
        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]

            # Evitamos duplicados
            if id_album in albumes_ya_vistos:
                continue
            albumes_ya_vistos.add(id_album)

            canciones_album = obtener_canciones_de_album(spotify, id_album)
            for cancion in canciones_album:
                # Guardamos info clara
                canciones.append({
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                })

    return canciones


In [14]:
# ────────────────────────────────────────────────
# MOSTRAR RESUMEN FINAL
# ────────────────────────────────────────────────
def mostrar_resumen(canciones, generos):
    print("RESUMEN:")
    for genero in generos:
        contador = sum(1 for c in canciones if c["genero"] == genero)
        print(f"- {genero}: {contador} canciones")

In [15]:
# ────────────────────────────────────────────────
# FUNCIÓN PRINCIPAL
# ────────────────────────────────────────────────
def main(spotify):
    """
    Función principal que orquesta todo el flujo.
    """
    generos = ["flamenco", "latin", "jazz", "rock"]
    año = 2016
    todas_las_canciones = []
    albumes_ya_vistos = set()

    print("Buscando canciones del año", año)
    print()

    for genero in generos:
        print("Género:", genero)
        canciones_genero = procesar_genero(spotify, genero, año, albumes_ya_vistos)
        todas_las_canciones.extend(canciones_genero)
        print("  → Canciones de este género añadidas\n")

    mostrar_resumen(todas_las_canciones, generos)
    print("Total de canciones encontradas:", len(todas_las_canciones))

    # --- Crear DataFrame directamente dentro de la función ---
    if todas_las_canciones:
        df = pd.DataFrame(todas_las_canciones)
        print("\nPrimeras 5 canciones:")
        print(df.head())
        # Guardar a CSV opcional
        df.to_csv(f"canciones_{año}.csv", index=False)
        print(f"\n✅ DataFrame guardado como canciones_{año}.csv")
        return df
    else:
        print("❌ No se encontraron canciones.")
        return None


In [16]:
# Llamamos a la función principal para obtener el DataFrame
todas_las_canciones_df = main(spotify)

Buscando canciones del año 2016

Género: flamenco
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- flamenco: 0 canciones
- latin: 217 canciones
- jazz: 460 canciones
- rock: 402 canciones
Total de canciones encontradas: 1079

Primeras 5 canciones:
                                              nombre    artista  \
0                                              Diles  Bad Bunny   
1                                           Soy Peor  Bad Bunny   
2                                               1986  Bad Bunny   
3             The Sutton Songs: No. 1, After He Died      Beéle   
4  The Sutton Songs: No. 2, Cold Iron Bars of Winter      Beéle   

                      album genero   año  
0                     Diles  latin  2016  
1                  Soy Peor  latin  2016  
2                      1986  latin  2016  
3  Alan Beeler: Voc

## 4. Obtener información LAST.FM

In [17]:
# ────────────────────────────────────────────────
# FUNCIÓN PARA CONSULTAR INFO DE ARTISTA EN LAST.FM
# ────────────────────────────────────────────────
def busqueda_info_artista(artista, api_key):
    """
    Consulta resumen y oyentes de un artista en Last.fm
    """
    url = "http://ws.audioscrobbler.com/2.0/"
    params = {
        "method": "artist.getinfo",
        "artist": quote(artista),
        "api_key": api_key,
        "format": "json"
    }

    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()

        if "artist" in data:
            bio = data["artist"].get("bio", {}).get("summary", "").split("<a href")[0].strip()
            listeners = int(data["artist"].get("stats", {}).get("listeners", 0))
            return {"artista": artista, "bio_resumen": bio, "listeners": listeners, "consulta_exitosa": True}
        else:
            return {"artista": artista, "consulta_exitosa": False, "error_lastfm": "No encontrado"}

    except Exception as e:
        return {"artista": artista, "consulta_exitosa": False, "error_lastfm": str(e)}

In [19]:
# ────────────────────────────────────────────────
# GENERAR DATAFRAME DE ARTISTAS
# ────────────────────────────────────────────────
def generar_datos_lastfm(canciones_df, api_key):
    if not api_key:
        print("❌ ERROR: La clave de Last.fm no está configurada")
        return None

    artistas_unicos = canciones_df['artista'].unique()
    resultados = []

    # Bucle simple sobre cada artista
    for artista in artistas_unicos:
        print(f"Consultando {artista}...")
        info = busqueda_info_artista(artista, api_key)
        resultados.append(info)

    # Convertir a DataFrame
    df_lastfm = pd.DataFrame(resultados)
    return df_lastfm

# Supongamos que tienes tu DataFrame de Spotify
# todas_las_canciones_df

api_key_lastfm = os.getenv("API_KEY_LASTFM") # Aseguramos que se usa la clave cargada

df_lastfm = generar_datos_lastfm(todas_las_canciones_df, api_key_lastfm)

print(df_lastfm.head())

Consultando Bad Bunny...
Consultando Beéle...
Consultando ROSALÍA...
Consultando Quevedo...
Consultando Mora...
Consultando Myke Towers...
Consultando Anuel AA...
Consultando Rauw Alejandro...
Consultando Frank Sinatra...
Consultando Robbie Williams...
Consultando Nat King Cole...
Consultando Norah Jones...
Consultando Dean Martin...
Consultando Louis Armstrong...
Consultando Ella Fitzgerald...
Consultando Bing Crosby...
Consultando Imagine Dragons...
Consultando The Beatles...
Consultando Radiohead...
Consultando Queen...
Consultando Maná...
Consultando Maroon 5...
     artista                                        bio_resumen  listeners  \
0  Bad Bunny  Benito Antonio Martínez Ocasio (born March 10,...    2122954   
1      Beéle  Brandon De Jesús López Orozco, mejor conocido ...     189238   
2    ROSALÍA  Rosalia Vila Tobella, known mononymously as RO...    1740378   
3    Quevedo  Pedro Luis Domínguez Quevedo es un cantante de...     362172   
4       Mora  There are at least two 

# 5. CREACIÓN DE LA BASE DE DATOS

In [20]:
# A. Conexión a MySQL
try:
    # Intentamos conectarnos al servidor MySQL
    cnx = mysql.connector.connect(
        host='127.0.0.1',    # Dirección del servidor MySQL (localhost)
        user='root',         # Usuario con el que nos conectamos
        password='DataBase2025#',  # Contraseña del usuario
        use_pure=True,       # use_pure=True garantiza compatibilidad con Python 3.12, evita problemas en el Kernel
    )
    print('Conexión exitosa')  # Confirmamos que la conexión fue correcta
except mysql.connector.Error as e:  # Captura errores relacionados con MySQL
    print('Error al conectar:', e)   # Muestra el error para diagnosticar el problema



Conexión exitosa


In [23]:
# B. Creación de la base de datos

try:
    # Creamos un cursor: es el intermediario entre Python y MySQL, necesario para ejecutar consultas SQL
    mycursor = cnx.cursor()

    # Consulta SQL para crear la base de datos si no existe
    # "IF NOT EXISTS" evita errores si la base de datos ya está creada
    query_create_database = "CREATE DATABASE IF NOT EXISTS MusicStream_db"

    # Ejecutamos la consulta SQL para crear la base de datos
    mycursor.execute(query_create_database)

    # Seleccionamos explícitamente la DB para las operaciones futuras (Creación de tablas e inserción)
    mycursor.execute("USE MusicStream_db")

    print("Base de datos creada y seleccionada correctamente.")

# Captura errores relacionados con MySQL (por ejemplo, permisos insuficientes)
except mysql.connector.Error as e:  
    print("Ocurrió un error al crear la base de datos:", e)


Base de datos creada y seleccionada correctamente.


In [24]:
# C. Creación de las tablas

# ------------------ Tabla Canciones ------------------
query_canciones = '''CREATE TABLE IF NOT EXISTS Canciones(
    ID_Cancion INT AUTO_INCREMENT,
    Nombre TEXT NOT NULL,
    Artista VARCHAR(45),
    Album TEXT,
    Genero VARCHAR(30),
    Lanzamiento YEAR,
    PRIMARY KEY (ID_Cancion)
);'''

try:
    mycursor.execute(query_canciones)
    print("Tabla 'Canciones' creada correctamente (o ya existía).")
except mysql.connector.Error as e:
    print("Ocurrió un error al crear la tabla 'Canciones':", e)
    
# ------------------ Tabla Artistas ------------------
query_artistas = '''CREATE TABLE IF NOT EXISTS Artistas(
    ID_Artista INT AUTO_INCREMENT,
    Artista VARCHAR(70) UNIQUE NOT NULL,
    Info TEXT,
    Oyentes INT,
    PRIMARY KEY (ID_Artista)
);'''

try:
    mycursor.execute(query_artistas)
    print("Tabla 'Artistas' creada correctamente (o ya existía).")
except mysql.connector.Error as e:
    print("Ocurrió un error al crear la tabla 'Artistas':", e)

Tabla 'Canciones' creada correctamente (o ya existía).
Tabla 'Artistas' creada correctamente (o ya existía).


# 6. INSERCIÓN DE LA BASE DE DATOS

In [ ]:
# Inserción de datos en la tabla 'Canciones'

# Seleccionamos la base de datos (redundante si se hizo en la Celda B, pero seguro)
mycursor.execute("USE MusicStream_db")

# Consulta de inserción de canciones
query_insert_canciones = """
INSERT INTO Canciones (Nombre, Artista, Album, Genero, Lanzamiento) 
VALUES (%s, %s, %s, %s, %s)
"""

try:
    # 1. Preparar datos del DataFrame de Spotify
    # Reemplazamos valores nulos (NaN) por None
    df_corregido_SPOTIFY = todas_las_canciones_df.replace({np.nan: None, 'nan': None, 'NaN': None})
    
    # 2. Filtrado de Columnas: Seleccionamos solo las 5 columnas que coinciden con la tabla SQL
    # Las columnas del DataFrame son 'nombre', 'artista', 'album', 'genero', 'año'
    datos = df_corregido_SPOTIFY[['nombre', 'artista', 'album', 'genero', 'año']].values.tolist()

    # 3. Ejecutar la inserción masiva
    mycursor.executemany(query_insert_canciones, datos)
    
    print(f"✅ {mycursor.rowcount} registros de canciones insertados correctamente.")
    
    # 4. Commit: Usamos 'cnx' que es la variable de conexión definida
    cnx.commit()  # Guardamos los cambios
    

except Error as e:
    print("❌ Error al insertar los datos de Spotify:", e)
    # Rollback: Usamos 'cnx' en caso de error
    cnx.rollback()  # Revertimos la operación en caso de error

✅ 1079 registros de canciones insertados correctamente.
